# 类设计

## is-a关系

In [1]:
class Employee:
    def __init__(self, name, salary=0):
        self.name = name
        self.salary = salary

    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)

    def work(self):
        print(self.name, "does stuff")

    def __repr__(self):
        return "<Employee: name=%s, salary=%s>" % (self.name, self.salary)


class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)

    def work(self):
        print(self.name, "makes food")


class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)

    def work(self):
        print(self.name, "interfaces with customer")


class PizzaRobot(Chef):
    def __init__(self, name):
        Chef.__init__(self, name)

    def work(self):
        print(self.name, "makes pizza")

In [2]:
bob = PizzaRobot('bob')  # Make a robot named bob
print(bob)  # Run inherited __repr__
bob.work()  # Run type-specific action
bob.giveRaise(0.20)  # Give bob a 20% raise
print(bob);
print()

for klass in Employee, Chef, Server, PizzaRobot:
    obj = klass(klass.__name__)
    obj.work()


<Employee: name=bob, salary=50000>
bob makes pizza
<Employee: name=bob, salary=60000.0>

Employee does stuff
Chef makes food
Server interfaces with customer
PizzaRobot makes pizza


## has-a关系

In [3]:
class Customer:
    def __init__(self, name):
        self.name = name

    def order(self, server):
        print(self.name, "orders from", server)

    def pay(self, server):
        print(self.name, "pays for item to", server)


class Oven:
    def bake(self):
        print("oven bakes")


class PizzaShop:
    def __init__(self):
        self.server = Server('Pat')  # Embed other objects
        self.chef = PizzaRobot('Bob')  # A robot named bob
        self.oven = Oven()

    def order(self, name):
        customer = Customer(name)  # Activate other objects
        customer.order(self.server)  # Customer orders from server
        self.chef.work()
        self.oven.bake()
        customer.pay(self.server)

In [4]:
scene = PizzaShop()  # Make the composite
scene.order('Homer')  # Simulate Homer's order
print('...')
scene.order('Shaggy')  # Simulate Shaggy's order


Homer orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Homer pays for item to <Employee: name=Pat, salary=40000>
...
Shaggy orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Shaggy pays for item to <Employee: name=Pat, salary=40000>


In [5]:
class Processor:
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer

    def process(self):
        while True:
            data = self.reader.read()
            if not data:
                break
            result = self.converter(data)
            self.writer.write(result)

    def converter(self, data):
        assert False, 'Should be implemented in subclass'


class Uppercase(Processor):
    def converter(self, data):
        return data.upper()

In [6]:
import sys

obj = Uppercase(open('text.log', 'r'), sys.stdout)

In [7]:
obj.process()

HELLO
WORLD
1 2 3


In [8]:
obj2 = Uppercase(open('text.log', 'r'), open('text_upper.txt', 'w'))
obj2.process()

In [9]:
class HTMLize:
    def write(self, line: str):
        print('<pre>', line.rstrip(), '</pre>')


In [10]:
Uppercase(open('text.log', 'r'), HTMLize()).process()

<pre> HELLO
WORLD
1 2 3 </pre>


In [11]:
import shelve

db = shelve.open('pizzashop.db')
db['key'] = PizzaShop()
db.close()

In [12]:
db = shelve.open('pizzashop.db')
shop = db['key']
shop.order('Tim')

Tim orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Tim pays for item to <Employee: name=Pat, salary=40000>


# 委托

In [13]:
class Wrapper:
    def __init__(self, obj):
        self.wrapped = obj

    def __getattr__(self, attrname):
        print('Trace:', attrname)
        return getattr(self.wrapped, attrname)

In [14]:
a = Wrapper([1, 2, 3])

In [15]:
getattr(a, 'append')

Trace: append


<function list.append(object, /)>

In [16]:
a.append(4)

Trace: append


In [17]:
a.wrapped

[1, 2, 3, 4]

In [18]:
x = Wrapper({'a': 1, 'b': 2})

In [19]:
x.keys()

Trace: keys


dict_keys(['a', 'b'])

In [20]:
list(x.keys())

Trace: keys


['a', 'b']

In [21]:
class Test1:
    __X = 'hello'

    def __method(self):
        print('hello')

In [22]:
a = Test1()
a.__X

AttributeError: 'Test1' object has no attribute '__X'

In [ ]:
a._Test1__X

In [ ]:
a.__method()

In [ ]:
a._Test1__method()

In [ ]:
class Test2:
    _X = 'hello'

    def _method(self):
        print('hello')

In [ ]:
b = Test2()
b._X

# 伪私有属性

In [23]:
class C1:
    def meth1(self): self.X = 88

    def meth2(self): print(self.X)


class C2:
    def metha(self): self.X = 99

    def methb(self): print(self.X)


class C3(C1, C2): ...

In [24]:
i = C3()

In [25]:
i.meth1(), i.metha()

(None, None)

In [26]:
i.__dict__

{'X': 99}

In [27]:
i.meth2(), i.methb()

99
99


(None, None)

In [28]:
class C1:
    def meth1(self): self.__X = 88

    def meth2(self): print(self.__X)


class C2:
    def metha(self): self.__X = 99

    def methb(self): print(self.__X)


class C3(C1, C2): ...

In [29]:
i = C3()
i.meth1(), i.metha()

(None, None)

In [30]:
i.__dict__

{'_C1__X': 88, '_C2__X': 99}

In [31]:
i.meth2(), i.methb()

88
99


(None, None)

# 绑定

In [32]:
class Spam:
    def doit(self, message):
        print(message)

In [33]:
a = Spam()
a.doit('hello')

hello


In [34]:
b = Spam()
x = b.doit
x('hello')

hello


In [35]:
c = Spam()
d = Spam.doit
d(c, 'hello')

hello


In [36]:
class Number:
    def __init__(self, base):
        self.base = base

    def double(self):
        return self.base * 2

    def triple(self):
        return self.base * 3

In [37]:
x = Number(2)
y = Number(3)
z = Number(4)

In [38]:
x.double()

4

In [39]:
acts = [x.double, y.double, z.double, z.triple]
for act in acts:
    print(act())

4
6
8
12


In [40]:
func = x.double

In [41]:
func.__self__

In [42]:
func.__func__

<function __main__.Number.double(self)>

In [43]:
func.__func__(func.__self__)

4

In [44]:
func.__self__.base

2

# 工厂函数

In [45]:
def factory(cls, *args, **kwargs):
    return cls(*args, **kwargs)


class Spam:
    def doit(self, message):
        print(message)


class Person:
    def __init__(self, name, job):
        self.name = name
        self.job = job

In [46]:
obj1 = factory(Spam)
obj2 = factory(Person, 'Bob', 'dev')
obj3 = factory(Person, job='manager', name='Tim')

In [47]:
obj1.doit('hello')

hello


In [48]:
obj2.name, obj3.job

('Bob', 'manager')

In [49]:
class ListInstance:
    """
    Mix-in class that provides a formatted print() or str() of instances via 
    inheritance of __str__ coded here;  displays instance attrs only;  self is
    instance of lowest class; __X names avoid clashing with client's attrs
    """

    def __attrnames(self):
        result = ''
        for attr in sorted(self.__dict__):
            result += f'\t{attr}={self.__dict__[attr]}\n'
        return result

    def __str__(self):
        return (f'<Instance of {self.__class__.__name__}'
                f'({self.__supers()}), '
                f'address {id(self)}:\n{self.__attrnames()}>')

    def __supers(self):
        names = []
        for super in self.__class__.__bases__:  # One level up from class
            names.append(super.__name__)  # name, not str(super)
        return ', '.join(names)
    # Or: ', '.join(super.__name__ for super in self.__class__.__bases__)

In [50]:
class Person(ListInstance):
    def __init__(self, name, job):
        self.name = name
        self.job = job

In [51]:
p = Person('Bob', 'dev')
print(p)

<Instance of Person(ListInstance), address 4613285520:
	job=dev
	name=Bob
>


In [52]:
str(p)

'<Instance of Person(ListInstance), address 4613285520:\n\tjob=dev\n\tname=Bob\n>'

In [53]:
p

In [54]:

class Super:
    def __init__(self):  # Superclass __init__
        self.data1 = 'spam'  # Create instance attrs

    def ham(self):
        pass


class Sub(Super, ListInstance):  # Mix in ham and a __str__
    def __init__(self):  # Listers have access to self
        Super.__init__(self)
        self.data2 = 'eggs'  # More instance attrs
        self.data3 = 42

    def spam(self):  # Define another method here
        pass

In [55]:
x = Sub()
print(x)

<Instance of Sub(Super, ListInstance), address 4645123152:
	data1=spam
	data2=eggs
	data3=42
>


In [56]:
class ListInherited:
    """
    Use dir() to collect both instance attrs and names inherited from 
    its classes;  Python 3.X shows more names than 2.X because of the 
    implied object superclass in the new-style class model;  getattr() 
    fetches inherited names not in self.__dict__;  use __str__, not 
    __repr__, or else this loops when printing bound methods!
    """
    def __attrnames(self, indent = 4*' '):
        under = []
        other = []
        for attr in dir(self):                              # Instance dir()
            if attr.startswith('__') and attr.endswith('__'):      # Skip internals
                under.append(attr)
            else:
                other.append(f'{indent}{attr}={getattr(self, attr)}')
        result = (str.center('Under', 77,'-')+ '\n' + indent + ', '.join(under) + 
                  '\n' + str.center('Other', 77,'-') + '\n' + '\n'.join(other))
        return result

    def __str__(self):
        return (f'<Instance of {self.__class__.__name__}, '
                f'address {id(self)}:\n{self.__attrnames()}>')


In [57]:
class Person(ListInherited):
    def __init__(self, name, job):
        self.name = name
        self.job = job
        
    def doit(self):
        pass
    
    

In [58]:
y = Person('Bob', 'dev')
print(y)

<Instance of Person, address 4645124496:
------------------------------------Under------------------------------------
    __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __getstate__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__
------------------------------------Other------------------------------------
    _ListInherited__attrnames=<bound method ListInherited.__attrnames of <__main__.Person object at 0x114def990>>
    doit=<bound method Person.doit of <__main__.Person object at 0x114def990>>
    job=dev
    name=Bob>


In [59]:
class Super:
    def __init__(self):  # Superclass __init__
        self.data1 = 'spam'  # Create instance attrs

    def ham(self):
        pass


class Sub(Super, ListInherited):  # Mix in ham and a __str__
    def __init__(self):  # Listers have access to self
        Super.__init__(self)
        self.data2 = 'eggs'  # More instance attrs
        self.data3 = 42

    def spam(self):  # Define another method here
        pass

In [60]:
x = Sub()
print(x)

<Instance of Sub, address 4645124048:
------------------------------------Under------------------------------------
    __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __getstate__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__
------------------------------------Other------------------------------------
    _ListInherited__attrnames=<bound method ListInherited.__attrnames of <__main__.Sub object at 0x114def7d0>>
    data1=spam
    data2=eggs
    data3=42
    ham=<bound method Super.ham of <__main__.Sub object at 0x114def7d0>>
    spam=<bound method Sub.spam of <__main__.Sub object at 0x114def7d0>>>


In [61]:
class ListTree:
    """
    Mix-in that returns an __str__ trace of the entire class tree and all 
    its objects' attrs at and above self;  run by print(), str() returns 
    constructed string;  uses __X attr names to avoid impacting clients;  
    recurses to superclasses explicitly, uses str.format() for clarity;
    """
    def __attrnames(self, obj, indent):
        spaces = ' ' * (indent + 1)
        result = ''
        for attr in sorted(obj.__dict__):
            if attr.startswith('__') and attr.endswith('__'):
                result += spaces + f'{attr}\n'
            else:
                result += spaces + f'{attr}={getattr(obj, attr)}\n'
        return result

    def __listclass(self, aClass, indent):
        dots = '.' * indent
        # print(f'{self.__visited=}')
        if aClass in self.__visited:
            return f'\n{dots}<Class {aClass.__name__}:, address {id(aClass)}: (see above)>\n'
        else:
            self.__visited[aClass] = True
            here  = self.__attrnames(aClass, indent)
            above = ''
            for super in aClass.__bases__:
                above += self.__listclass(super, indent+4)
            return f'\n{dots}<Class {aClass.__name__}, address {id(aClass)}:\n{here}{above}{dots}>\n'

    def __str__(self):
        self.__visited = {}
        here  = self.__attrnames(self, 0)
        above = self.__listclass(self.__class__, 4)
        return f'<Instance of {self.__class__.__name__}, address {id(self)}:\n{here}{above}>'



In [62]:
class Super:
    def __init__(self):  # Superclass __init__
        self.data1 = 'spam'  # Create instance attrs

    def ham(self):
        pass


class Sub(Super, ListTree):  # Mix in ham and a __str__
    def __init__(self):  # Listers have access to self
        Super.__init__(self)
        self.data2 = 'eggs'  # More instance attrs
        self.data3 = 42

    def spam(self):  # Define another method here
        pass

In [63]:
x = Sub()
print(x)

<Instance of Sub, address 4645352208:
 _ListTree__visited={}
 data1=spam
 data2=eggs
 data3=42

....<Class Sub, address 4402767728:
     __doc__
     __init__
     __module__
     spam=<function Sub.spam at 0x114dd56c0>

........<Class Super, address 4402766000:
         __dict__
         __doc__
         __init__
         __module__
         __weakref__
         ham=<function Super.ham at 0x114dd5800>

............<Class object, address 4379248016:
             __class__
             __delattr__
             __dir__
             __doc__
             __eq__
             __format__
             __ge__
             __getattribute__
             __getstate__
             __gt__
             __hash__
             __init__
             __init_subclass__
             __le__
             __lt__
             __ne__
             __new__
             __reduce__
             __reduce_ex__
             __repr__
             __setattr__
             __sizeof__
             __str__
             __su

In [64]:
from PySide6.QtCore import QAbstractItemModel

In [65]:
class MyButton(QAbstractItemModel, ListTree):
    def doit(self):
        print('doit')

In [66]:
a = MyButton()
print(a)

<Instance of MyButton, address 4868680320:
 _ListTree__visited={}
 columnsAboutToBeInserted=<PySide6.QtCore.SignalInstance columnsAboutToBeInserted(QModelIndex,int,int) at 0x12228e6b0>
 columnsAboutToBeMoved=<PySide6.QtCore.SignalInstance columnsAboutToBeMoved(QModelIndex,int,int,QModelIndex,int) at 0x12228e630>
 columnsAboutToBeRemoved=<PySide6.QtCore.SignalInstance columnsAboutToBeRemoved(QModelIndex,int,int) at 0x12228e6f0>
 columnsInserted=<PySide6.QtCore.SignalInstance columnsInserted(QModelIndex,int,int) at 0x12228e5f0>
 columnsMoved=<PySide6.QtCore.SignalInstance columnsMoved(QModelIndex,int,int,QModelIndex,int) at 0x12228e2d0>
 columnsRemoved=<PySide6.QtCore.SignalInstance columnsRemoved(QModelIndex,int,int) at 0x12228e4f0>
 dataChanged=<PySide6.QtCore.SignalInstance dataChanged(QModelIndex,QModelIndex,QList<int>) at 0x12228e090>
 destroyed=<PySide6.QtCore.SignalInstance destroyed() at 0x12228e770>
 headerDataChanged=<PySide6.QtCore.SignalInstance headerDataChanged(Qt::Orientat

In [67]:
b = MyButton()
print(b)

<Instance of MyButton, address 4868684928:
 _ListTree__visited={}
 columnsAboutToBeInserted=<PySide6.QtCore.SignalInstance columnsAboutToBeInserted(QModelIndex,int,int) at 0x12228ed70>
 columnsAboutToBeMoved=<PySide6.QtCore.SignalInstance columnsAboutToBeMoved(QModelIndex,int,int,QModelIndex,int) at 0x12228ecf0>
 columnsAboutToBeRemoved=<PySide6.QtCore.SignalInstance columnsAboutToBeRemoved(QModelIndex,int,int) at 0x12228edb0>
 columnsInserted=<PySide6.QtCore.SignalInstance columnsInserted(QModelIndex,int,int) at 0x12228ecb0>
 columnsMoved=<PySide6.QtCore.SignalInstance columnsMoved(QModelIndex,int,int,QModelIndex,int) at 0x12228e970>
 columnsRemoved=<PySide6.QtCore.SignalInstance columnsRemoved(QModelIndex,int,int) at 0x12228ebb0>
 dataChanged=<PySide6.QtCore.SignalInstance dataChanged(QModelIndex,QModelIndex,QList<int>) at 0x12228e8d0>
 destroyed=<PySide6.QtCore.SignalInstance destroyed() at 0x12228ee30>
 headerDataChanged=<PySide6.QtCore.SignalInstance headerDataChanged(Qt::Orientat